# Import necessary packages

In [45]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [46]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import original data and preprocessed data

In [47]:
#sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
#calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

In [48]:
sales_train_val = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\sales_train_validation.csv')
calendar = pd.read_csv(r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\M5 Forecasting\Estimate the unit sales of Walmart retail goods\m5-forecasting-accuracy_unzip files\calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [49]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [50]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [51]:
INPUT_DIR_2 = r'C:\Users\ASUS\Desktop\Academic Work\Academic Research\DEMAND PATTERN\CRUNCH TIME (WEEKLY AGGREGATION)\California\Foods\Demand Pattern Categorization'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [52]:
len(sales_intermittent)

1301

In [53]:
len(sales_lumpy)

395

In [54]:
len(sales_erratic)

458

In [55]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [56]:
sales_erratic

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1665,FOODS_1_055_CA_1_validation,FOODS_1_055,FOODS_1,FOODS,CA_1,CA,2,5,2,5,...,0,0,1,2,3,12,5,1,1,6
1669,FOODS_1_059_CA_1_validation,FOODS_1_059,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1678,FOODS_1_068_CA_1_validation,FOODS_1_068,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1692,FOODS_1_082_CA_1_validation,FOODS_1_082,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,4,14,8,6,9,10,10,12,8,7
1712,FOODS_1_103_CA_1_validation,FOODS_1_103,FOODS_1,FOODS,CA_1,CA,7,1,1,2,...,1,0,1,2,1,1,3,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,FOODS_3_777_CA_4_validation,FOODS_3_777,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,4,1,1,0,1,0,0,0,0,0
12169,FOODS_3_801_CA_4_validation,FOODS_3_801,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,3,2,2,0,0,0,1,2,3
12177,FOODS_3_809_CA_4_validation,FOODS_3_809,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,1,1,1,0,7,1,2,4,0
12179,FOODS_3_811_CA_4_validation,FOODS_3_811,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,2,7,4,1,0,0,2,3,5,2


# User-defined functions to calculate Metrics

In [57]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)


def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

# Format sales data and then merge with calendar data

In [58]:
sales_pattern = sales_erratic.copy()

In [59]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [60]:
dataframe_weekly

,wm_yr_wk,FOODS_1_003_CA_4,FOODS_1_004_CA_2,FOODS_1_019_CA_3,FOODS_1_022_CA_3,FOODS_1_025_CA_4,FOODS_1_027_CA_3,FOODS_1_030_CA_3,FOODS_1_032_CA_3,FOODS_1_034_CA_2,...,FOODS_3_809_CA_4,FOODS_3_811_CA_2,FOODS_3_811_CA_4,FOODS_3_815_CA_3,FOODS_3_816_CA_3,FOODS_3_816_CA_4,FOODS_3_822_CA_2,FOODS_3_822_CA_3,FOODS_3_825_CA_3,FOODS_3_827_CA_3
0,11101,2.0,15.0,30.0,3.0,5.0,12.0,3.0,28.0,1.0,...,0.0,18.0,19.0,0.0,31.0,7.0,22.0,29.0,0.0,0.0
1,11102,7.0,31.0,9.0,10.0,4.0,23.0,1.0,18.0,0.0,...,9.0,31.0,29.0,0.0,55.0,7.0,14.0,36.0,3.0,0.0
2,11103,6.0,28.0,0.0,11.0,6.0,14.0,3.0,17.0,0.0,...,3.0,25.0,31.0,0.0,117.0,3.0,27.0,50.0,7.0,0.0
3,11104,3.0,37.0,2.0,3.0,4.0,31.0,4.0,20.0,1.0,...,9.0,15.0,19.0,0.0,49.0,10.0,16.0,37.0,3.0,0.0
4,11105,7.0,40.0,0.0,8.0,4.0,24.0,3.0,17.0,2.0,...,3.0,0.0,0.0,1.0,61.0,5.0,17.0,34.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,2.0,39.0,50.0,3.0,6.0,17.0,2.0,23.0,3.0,...,11.0,22.0,19.0,0.0,64.0,14.0,11.0,37.0,19.0,0.0
269,11609,23.0,47.0,3.0,4.0,2.0,18.0,0.0,21.0,2.0,...,6.0,34.0,22.0,0.0,83.0,10.0,14.0,39.0,5.0,0.0
270,11610,2.0,56.0,6.0,4.0,4.0,22.0,1.0,22.0,0.0,...,4.0,32.0,14.0,0.0,22.0,19.0,28.0,52.0,0.0,0.0
271,11611,2.0,31.0,11.0,10.0,5.0,14.0,2.0,43.0,1.0,...,4.0,34.0,15.0,0.0,59.0,3.0,18.0,37.0,3.0,0.0


In [61]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

# Run ARIMA model with best combination

In [62]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [65]:
len(list_pattern_py)

458

In [63]:
best_combination = (0, 1, 2)

In [64]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    
    train_product = train_data[product]
    test_product = test_data[product]
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()
    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Product': [product for product_count in range(len(test_product))],
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_055_CA_1
Progressing: 0.0 %
Currently Running Product: FOODS_1_059_CA_1
Progressing: 0.22 %
Currently Running Product: FOODS_1_068_CA_1
Progressing: 0.44 %
Currently Running Product: FOODS_1_082_CA_1
Progressing: 0.66 %
Currently Running Product: FOODS_1_103_CA_1
Progressing: 0.87 %
Currently Running Product: FOODS_1_113_CA_1
Progressing: 1.09 %
Currently Running Product: FOODS_1_115_CA_1
Progressing: 1.31 %
Currently Running Product: FOODS_1_128_CA_1
Progressing: 1.53 %
Currently Running Product: FOODS_1_131_CA_1
Progressing: 1.75 %
Currently Running Product: FOODS_1_132_CA_1
Progressing: 1.97 %
Currently Running Product: FOODS_1_163_CA_1
Progressing: 2.18 %
Currently Running Product: FOODS_1_171_CA_1
Progressing: 2.4 %
Currently Running Product: FOODS_1_179_CA_1
Progressing: 2.62 %
Currently Running Product: FOODS_1_180_CA_1
Progressing: 2.84 %
Currently Running Product: FOODS_1_216_CA_1
Progressing: 3.06 %
Currently Running Product: FOODS_2_003_CA_

Currently Running Product: FOODS_3_207_CA_2
Progressing: 28.17 %
Currently Running Product: FOODS_3_212_CA_2
Progressing: 28.38 %
Currently Running Product: FOODS_3_245_CA_2
Progressing: 28.6 %
Currently Running Product: FOODS_3_263_CA_2
Progressing: 28.82 %
Currently Running Product: FOODS_3_268_CA_2
Progressing: 29.04 %
Currently Running Product: FOODS_3_306_CA_2
Progressing: 29.26 %
Currently Running Product: FOODS_3_311_CA_2
Progressing: 29.48 %
Currently Running Product: FOODS_3_313_CA_2
Progressing: 29.69 %
Currently Running Product: FOODS_3_321_CA_2
Progressing: 29.91 %
Currently Running Product: FOODS_3_333_CA_2
Progressing: 30.13 %
Currently Running Product: FOODS_3_353_CA_2
Progressing: 30.35 %
Currently Running Product: FOODS_3_376_CA_2
Progressing: 30.57 %
Currently Running Product: FOODS_3_432_CA_2
Progressing: 30.79 %
Currently Running Product: FOODS_3_433_CA_2
Progressing: 31.0 %
Currently Running Product: FOODS_3_437_CA_2
Progressing: 31.22 %
Currently Running Product: 

Currently Running Product: FOODS_3_004_CA_3
Progressing: 56.55 %
Currently Running Product: FOODS_3_021_CA_3
Progressing: 56.77 %
Currently Running Product: FOODS_3_024_CA_3
Progressing: 56.99 %
Currently Running Product: FOODS_3_044_CA_3
Progressing: 57.21 %
Currently Running Product: FOODS_3_065_CA_3
Progressing: 57.42 %
Currently Running Product: FOODS_3_066_CA_3
Progressing: 57.64 %
Currently Running Product: FOODS_3_069_CA_3
Progressing: 57.86 %
Currently Running Product: FOODS_3_077_CA_3
Progressing: 58.08 %
Currently Running Product: FOODS_3_078_CA_3
Progressing: 58.3 %
Currently Running Product: FOODS_3_084_CA_3
Progressing: 58.52 %
Currently Running Product: FOODS_3_086_CA_3
Progressing: 58.73 %
Currently Running Product: FOODS_3_094_CA_3
Progressing: 58.95 %
Currently Running Product: FOODS_3_100_CA_3
Progressing: 59.17 %
Currently Running Product: FOODS_3_104_CA_3
Progressing: 59.39 %
Currently Running Product: FOODS_3_106_CA_3
Progressing: 59.61 %
Currently Running Product:

Currently Running Product: FOODS_3_822_CA_3
Progressing: 84.28 %
Currently Running Product: FOODS_3_825_CA_3
Progressing: 84.5 %
Currently Running Product: FOODS_3_827_CA_3
Progressing: 84.72 %
Currently Running Product: FOODS_1_003_CA_4
Progressing: 84.93 %
Currently Running Product: FOODS_1_025_CA_4
Progressing: 85.15 %
Currently Running Product: FOODS_1_055_CA_4
Progressing: 85.37 %
Currently Running Product: FOODS_1_058_CA_4
Progressing: 85.59 %
Currently Running Product: FOODS_1_088_CA_4
Progressing: 85.81 %
Currently Running Product: FOODS_1_095_CA_4
Progressing: 86.03 %
Currently Running Product: FOODS_1_120_CA_4
Progressing: 86.24 %
Currently Running Product: FOODS_1_132_CA_4
Progressing: 86.46 %
Currently Running Product: FOODS_1_143_CA_4
Progressing: 86.68 %
Currently Running Product: FOODS_1_163_CA_4
Progressing: 86.9 %
Currently Running Product: FOODS_1_164_CA_4
Progressing: 87.12 %
Currently Running Product: FOODS_1_190_CA_4
Progressing: 87.34 %
Currently Running Product: 

# Calculate metrics for optimal combination

In [67]:
# Start calculating metrics 
start = timer()

df_arima_metrics = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.01178172166667082 minutes


In [69]:
df_arima_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_003_CA_4,0.511584,65.1107,71.1484,119.1997,3.581088e+00,3.616220e+00
1,FOODS_1_004_CA_2,0.755710,34.0070,31.1683,41.4271,1.360279e+01,1.657054e+01
2,FOODS_1_019_CA_3,1.703918,340.7836,139.9500,133.0557,1.874310e+01,1.918778e+01
3,FOODS_1_022_CA_3,0.498309,99.6618,147.6976,64.5882,4.983091e+00,5.197016e+00
4,FOODS_1_025_CA_4,inf,3.8340,3.9552,3.8340,1.916999e-01,2.449921e-01
...,...,...,...,...,...,...,...
453,FOODS_3_816_CA_4,0.540211,73.9236,75.0009,194.5872,7.022741e+00,8.134745e+00
454,FOODS_3_822_CA_2,1.300259,86.6839,64.3025,107.7422,1.170233e+01,1.226738e+01
455,FOODS_3_822_CA_3,0.652453,19.4986,18.7778,22.2590,8.481892e+00,1.074047e+01
456,FOODS_3_825_CA_3,0.326923,29.7203,34.8086,49.3138,1.634616e+00,1.991720e+00


In [71]:
df_arima_metrics.MAPE.mean()

91.89261487603305

In [72]:
df_arima_metrics.RMSE.mean()

8.579718958483156

In [74]:
df_arima_metrics.to_csv('ARIMA_Erratic_Forecast_Metrics.csv')